In [57]:
# connect the google drive for inputs

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
# change dir to shared drive and list folders and files

import os

os.chdir("/content/drive/Shareddrives/")
!ls

'2020 EID Orientation'
'2021 2022 Course Syllabus'
'2022 MCO602 Molecular Parasitology'
'*2023-2024 Course Descriptions Schedules'
'Book Downloads'
'Class Files :)'
"Commencement Photo's"
'COVID 19 Immunology Journal Club'
'EID501 Models Course 20-21'
'EID502 Models Course 20-21'
'EID503 Models of Emerging Infectious Diseases Course'
'EID504 Models Course Spring 2022'
'GEO Intranet Files'
'GEO Student Information'
'Graduate Students @ USUHS'
'MCB0801  Techniques Spring 2021'
'MCB531: Intro Immunology (2020 Fall)'
'MCO503 Molecular and Cellular Immunology 2021-2022'
'Mitre Lab'
'SBI Luminex Project'
'Tonia Project Information'
'Virtual Lunch & Learn Sessions'


In [59]:
# setting up the program dir

shared_drive = "/content/drive/Shareddrives/"

prog_dir = 'SBI Luminex Project/'
prog_dir_path = shared_drive + prog_dir
print(prog_dir_path)
os.chdir(prog_dir_path)
!ls

/content/drive/Shareddrives/SBI Luminex Project/
'archive analysis files 2022'	     'Saliva 22NOV22'
 Master_Tables			     'Serum 10JAN23'
'Prior Files to Build Program 2022'  'TC program copies'
'Program Run Notes TC'


In [60]:
# import statements

import os
from google.cloud import storage
import pandas as pd
import numpy as np
from os.path import exists
from pathlib import Path
from os.path import exists
import re
import sys
from os import listdir
from datetime import date
import time
import scipy
from scipy.stats.mstats import gmean
from scipy import stats
import shutil, sys
import glob

In [61]:
# method to read write file
# input is sample file or standards file to split into instrument and data files based on number of lines to write into instrument file (instr_split_loc) 6, line number after instrument data line (data_split_loc)7
#############################################################################################

def create_dir (path):
    try:
        os.mkdir(path)
    except OSError as error:
        print(error)
#############################################################################################

# method to find today's date and time
today = date.today()
today
# dd/mm/YY
today_date = today.strftime("%d%m%Y")
print("today_date =", today_date)

curr_time = time.strftime("%H:%M:%S", time.localtime())

print("Current Time is :", curr_time)
#############################################################################################
def read_csv_split_instrument_and_data_df(ip_file, instr_split_loc, data_split_loc):
  # read the result file into dataframe
  df = pd.read_excel(ip_file, header=None)#, skiprows = 6)
  df

  #first 6 rows and column 1 is required instrument info, separated from df to create insrument info csv file
  instr_info_df = df.iloc[:instr_split_loc,:1]#[:6,:1]

  #creating data file, required table
  dt_df = df.iloc[data_split_loc:,:]#[7:,:]

  # Convert Row to Column Header in DataFrame
  header_row = dt_df.iloc[0]
  dt_df = pd.DataFrame(dt_df.values[1:], columns=header_row)

  #drop samplingerror col
  dt_df = dt_df.drop('Sampling Errors', axis=1)

  # remove special character
  dt_df.columns = dt_df.columns.str.rsplit(' ').str[0]
  dt_df.columns = dt_df.columns.str.strip()

  return instr_info_df, dt_df

today_date = 29062023
Current Time is : 20:22:59


In [62]:
# Setup the paths and create analysys dir for today
serum_prog_dir = 'Serum 10JAN23'
serum_prog_dir_path = prog_dir_path + serum_prog_dir
print('serum_prog_dir_path: ', serum_prog_dir_path)

analysis_dir_name = serum_prog_dir.replace(' ', '') + '_analysis_' + today_date # replace space in the name
analysis_dir_name_path = serum_prog_dir_path + '/output/' + analysis_dir_name

print(analysis_dir_name_path)
create_dir(analysis_dir_name_path)


serum_prog_dir_path:  /content/drive/Shareddrives/SBI Luminex Project/Serum 10JAN23
/content/drive/Shareddrives/SBI Luminex Project/Serum 10JAN23/output/Serum10JAN23_analysis_29062023
[Errno 17] File exists: '/content/drive/Shareddrives/SBI Luminex Project/Serum 10JAN23/output/Serum10JAN23_analysis_29062023'


In [63]:
# create plates dir list
# problem with 'IgG plate plans, standards and sample results for merging' sample num column
# good 'SIgA plate plans, standards and sample results for merging','IgA plate plans, standards and sample results for merging',
plates_dir_list = ['IgG plate plans and sample results']
print(len(plates_dir_list))

1


In [64]:
# iterate through all plates and do process

for plate in plates_dir_list:
    print('plate: ', plate)
    ip_dir_path = serum_prog_dir_path + '/input/' + plate
    print('ip_dir_path: ', ip_dir_path)

    # create a name for output dir for each plate
    list = plate.replace(',', '').split(' ') # replace the comma in the name and split the plate name with white space
    print(list)
    opdir_name = list[0] + '_' + list[1] + '_' + list[2] + '_Output'
    print('opdir_name ', opdir_name)
    # create the output dir for each plate
    create_dir(analysis_dir_name_path + '/' + opdir_name)
    # create the output/archive dir for each plate
    create_dir(analysis_dir_name_path + '/' + opdir_name + '/' + 'archive')

    # List the files in the given dir and create sample, standard and plateplan file names
    dir_list = os.listdir(ip_dir_path)
    print(dir_list)

    for file in dir_list:
      print('file: ', file)
      if 'sample' in file:
        sample_result_file_name = file
        sample_result_file = ip_dir_path + '/' + sample_result_file_name
        print('sample_result_file: ', sample_result_file)
      elif 'standard' in file:
        standard_result_file_name = file
        standard_result_file = ip_dir_path + '/' + standard_result_file_name
        print('standard_result_file: ', standard_result_file)
      elif 'plate_plan' in file:
        plate_plan_file_name = file
        plate_plan_file = ip_dir_path + '/' + plate_plan_file_name
        print('plate_plan_file: ', plate_plan_file)
      else:
        print('Files not found')

        # read plateplan file into dataframe
    pp_df = pd.read_excel(plate_plan_file) #
    print('pp_df:', pp_df.dtypes)
    #pp_df = pp_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    pp_df['SampleNum'] = pp_df['SampleNum'].astype(int)

    print(pp_df.head())
    print(pp_df.dtypes)
    # processing sample file  ------------------------------------------------------------------------------------------------ Step 1 merge sample and plateplan data

    #call read_csv_split_instrument_and_data_df method to read csv file and get instrument and data dfs
    sm_instr_info_df, sm_data_df = read_csv_split_instrument_and_data_df(sample_result_file, 6, 7)

    print(sm_instr_info_df.head())
    print(sm_data_df.head())
    # write the instrument and data files into output dir
    sm_instr_info_df.to_csv(analysis_dir_name_path + '/' + opdir_name + '/' + sample_result_file_name.split('.', 1)[0] + '_instrument_info.csv',header=False, index=False)
    sm_data_df.to_csv(analysis_dir_name_path + '/' + opdir_name + '/' + sample_result_file_name.split('.', 1)[0] + '_data.csv', index=False)

    # merge sm data df and plateplan df
    merged_df_sm = pd.merge(sm_data_df,pp_df, on=['Well','Type'])
    print(merged_df_sm.head())

    # re-arrange the columns and create a new df *****
    # 'Well	Type	Mock	SCV2SP	SCV2NP	NL63	HKU1	229E	OC43	SampleNum	SampleID	CollectDate	DF' to 'SampleNum',	'SampleID','CollectDate', 'Well',	'Type', 'DF',	'Mock', 'SCV2SP',	'SCV2NP',	'NL63',	'HKU1',	'229E',	'OC43'
    ra_df_sm = merged_df_sm[['SampleNum',	'SampleID','CollectDate', 'Well',	'Type', 'DF', 'SCV2SP', 'SCV2NP',   'OC43' ,   'HKU1' ,   '229E' ,    'NL63']]

    # set samplenum as index
    ra_df_sm = ra_df_sm.set_index('SampleNum')
    print("ra_df_sm before memoving PBS rows: ", ra_df_sm.head())

    # remove rows where SampleID is equal to PBS
    ra_df_sm.drop(ra_df_sm.loc[ra_df_sm['SampleID']=='PBS'].index, inplace=True)
    print("ra_df after memoving PBS rows: ", ra_df_sm.head())

    # write the op into output dir
    ra_df_sm.to_csv(analysis_dir_name_path + '/' + opdir_name + '/' + sample_result_file_name.split('.', 1)[0] + '_merged_data.csv')

    #read Saliva Antibody Standard Concentrations file into dataframe
    resources_dir = serum_prog_dir_path + '/resources/'

    resources_dir_list = os.listdir(resources_dir)
    print(resources_dir_list)

    for resources_file in resources_dir_list:
      print('resources_file: ', resources_file)
      if 'Concentrations' in resources_file:
        serum_ab_conc_file = resources_dir + resources_file
      else:
        print('file not found!')

    serum_ab_conc_df = pd.read_excel(serum_ab_conc_file)
    print(serum_ab_conc_df.head())

    # map merged df with saliva conc df on type and get conc col
    IgG_Concentrations = ra_df_sm['Type'].map(serum_ab_conc_df.set_index('Type')['IgG Concentrations'])
    #ra_df_sm.insert(6, 'IgG Concentrations', IgG_Concentrations) with new protein columns change the number
    ra_df_sm.insert(11, 'IgG Concentrations', IgG_Concentrations)
    print(ra_df_sm.head())

    # write the op into output dir
    ra_df_sm.to_csv(analysis_dir_name_path + '/' + opdir_name + '/' + sample_result_file_name.split('.', 1)[0] + '_conc_merged_data.csv')

plate:  IgG plate plans and sample results
ip_dir_path:  /content/drive/Shareddrives/SBI Luminex Project/Serum 10JAN23/input/IgG plate plans and sample results
['IgG', 'plate', 'plans', 'and', 'sample', 'results']
opdir_name  IgG_plate_plans_Output
[Errno 17] File exists: '/content/drive/Shareddrives/SBI Luminex Project/Serum 10JAN23/output/Serum10JAN23_analysis_29062023/IgG_plate_plans_Output'
[Errno 17] File exists: '/content/drive/Shareddrives/SBI Luminex Project/Serum 10JAN23/output/Serum10JAN23_analysis_29062023/IgG_plate_plans_Output/archive'
['archive', 'Test file', 'ABCD_values_GP.xlsx', 'plate_plan_IGG_02_29JUN23.xlsx', 'sample_results_IGG_02_29JUN23.xlsx']
file:  archive
Files not found
file:  Test file
Files not found
file:  ABCD_values_GP.xlsx
Files not found
file:  plate_plan_IGG_02_29JUN23.xlsx
plate_plan_file:  /content/drive/Shareddrives/SBI Luminex Project/Serum 10JAN23/input/IgG plate plans and sample results/plate_plan_IGG_02_29JUN23.xlsx
file:  sample_results_IGG_02

In [65]:
# read the abcd file from input dir
abcd_file = ip_dir_path + '/' +'ABCD_values_GP.xlsx'
abcd_df = pd.read_excel(abcd_file)
abcd_df.columns = abcd_df.columns.str.strip()
abcd_df

,parameters,SCV2SP,HKU1,229E,OC43,NL63,SCV2NP
0,Bottom,825.20000,-3.521000e+02,1045.0000,385.40000,717.500,596.3000
1,Top,32015.00000,7.808466e+07,38547.0000,32300.00000,20294.000,31723.0000
2,IC50,0.08268,3.475600e+04,0.6375,0.07472,1.484,0.1524
3,HillSlope,2.03300,7.933000e-01,1.2270,1.90900,1.150,2.0180


In [66]:
#######################################################################################################################################################################################################
# interpolation steps

# curve fitting 4pl

print(ra_df_sm)

# iterate tp all proteins
def func_find_x (y, a, b, c, d):
    x = c*((((a-d)/(y-d)) -1)**(1/b))
    return x

# list the proteins
prot_list = ['SCV2SP',	'SCV2NP',	'NL63',	'HKU1',	'OC43','229E'	]

for i in prot_list:
  print(i)
  # create a required df
  df_req = ra_df_sm[['SampleID','CollectDate', 'DF',  i,'IgG Concentrations']]
  #print(df_req)

  # Step1
  #create train and test dataframes
  test_df = df_req[df_req['IgG Concentrations'].isnull()]
  #print('test_df' , test_df)

  # Step 2
  # test data from test df for y
  ytest = np.array(test_df[i])
  print(ytest)

  # Step 3
  # determining the a,b,c,d values from xdata and ydata # needs to be automated

  #copy the above x,y values into the following website and calculate X50 and get a,b,c,d values from formula
  #https://www.aatbio.com/tools/four-parameter-logistic-4pl-curve-regression-online-calculator

  # get abcd values from abcd_df ----- reading from a file created with above website or graphpad curvefit
  sub_abcd_df = abcd_df[['parameters',i]]
  print(sub_abcd_df)

  #a = sub_abcd_df.loc[sub_abcd_df['parameters'] == 'Bottom']
  #df.loc[df['B'] == 3, 'A'].iloc[0]
  a = sub_abcd_df.loc[sub_abcd_df['parameters'] == 'Bottom',i].iloc[0]
  b = sub_abcd_df.loc[sub_abcd_df['parameters'] == 'HillSlope',i].iloc[0]
  c = sub_abcd_df.loc[sub_abcd_df['parameters'] == 'IC50',i].iloc[0]
  d = sub_abcd_df.loc[sub_abcd_df['parameters'] == 'Top',i].iloc[0]
  print('a: ', a, ' b: ', b, 'c: ', c, ' d: ', d)

  # Step 4
  # function for calculating x based on y, a,b,c, and d

  #function for calculating concentrations from y values
  #https://github.com/Tom-Alexander/regression-js/issues/96


  # Step 5 to calulate the x_test

  # test data calculations
  x_test = func_find_x(ytest,a,b,c,d)
  print('x_test: ', x_test)

  # Step 6
  # adding the x_test values to test dataframe

  test_df['IgG Concentrations_interpolated'] = x_test
  print(test_df)

  print(test_df.columns)
  # write the op into output dir
  test_df.to_csv(analysis_dir_name_path + '/' + opdir_name + '/' + sample_result_file_name.split('.', 1)[0] + '_' + i + '_conc_interpolated.csv')

  # Step 7
  test_df['final'] = test_df['DF']*test_df['IgG Concentrations_interpolated']
  print(test_df.columns)

  # step 8 write the final output file
  # write the op into output dir
  # Remove 'SampleID','Well', 'Type','IgG Concentrations'
  test_df = test_df.drop(['IgG Concentrations'], axis=1)
  test_df.to_csv(analysis_dir_name_path + '/' + opdir_name + '/' + sample_result_file_name.split('.', 1)[0] + '_' + i + '_final_output.csv')


               SampleID CollectDate Well Type     DF   SCV2SP   SCV2NP  \
SampleNum                                                                
1            PASS 1:400         NaT   A1   S1    400    31760    30709   
2            PASS 1:800         NaT   B1   S2    800    31362    29255   
3           PASS 1:1.6k         NaT   C1   S3   1600  29335.5    23590   
4           PASS 1:3.2k         NaT   D1   S4   3200    23799  12060.5   
5           PASS 1:6.4k         NaT   E1   S5   6400    12660   5528.5   
6          PASS 1:12.8k         NaT   F1   S6  12800   5465.5     2283   
7          PASS 1:25.6k         NaT   G1   S7  25600     2270    621.5   
8          PASS 1:51.2k         NaT   H1   S8  51200      717    259.5   
10         Negative (-)         NaT   B2   C1    400   1459.5   4536.5   
11              1191.03  2021-01-08   C2   X1    400    32734     1593   
12              1194.13  2022-12-08   D2   X2    400    32737     6914   
13              1211.14  2022-12-01   

<ipython-input-66-ae36fe594ed5>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['IgG Concentrations_interpolated'] = x_test
<ipython-input-66-ae36fe594ed5>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['final'] = test_df['DF']*test_df['IgG Concentrations_interpolated']
<ipython-input-66-ae36fe594ed5>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [67]:
# Step1

#create train and test dataframes

train_df = ra_df_sm[ra_df_sm['IgG Concentrations'].notnull()]
print('train_df', train_df)

test_df = ra_df_sm[ra_df_sm['IgG Concentrations'].isnull()]
print('test_df' , test_df)

train_df                SampleID CollectDate Well Type     DF   SCV2SP   SCV2NP  \
SampleNum                                                                
1            PASS 1:400         NaT   A1   S1    400    31760    30709   
2            PASS 1:800         NaT   B1   S2    800    31362    29255   
3           PASS 1:1.6k         NaT   C1   S3   1600  29335.5    23590   
4           PASS 1:3.2k         NaT   D1   S4   3200    23799  12060.5   
5           PASS 1:6.4k         NaT   E1   S5   6400    12660   5528.5   
6          PASS 1:12.8k         NaT   F1   S6  12800   5465.5     2283   
7          PASS 1:25.6k         NaT   G1   S7  25600     2270    621.5   
8          PASS 1:51.2k         NaT   H1   S8  51200      717    259.5   

              OC43    HKU1    229E  NL63  IgG Concentrations  
SampleNum                                                     
1            31848   20278   25478  8669            1.070000  
2          31617.5   11300   18050  5434            0.535000 

In [68]:
# Step 2
# x,y data from train df

xdata = np.array(train_df['IgG Concentrations'])
print(xdata)

ydata = np.array(train_df['SCV2SP'])
print(ydata)

# test data from test df for y
ytest = np.array(test_df['SCV2SP'])
print(ytest)

[1.07       0.535      0.2675     0.13375    0.066875   0.0334375
 0.01671875 0.00835938]
[31760 31362 29335.5 23799 12660 5465.5 2270 717]
[1459.5 32734 32737 31094 32737.5 32735 32737 32738 32736 32737 32236
 32672 8560 30901.5 24868 27090 32646 31369.5 24177 32539 11322 2202.5
 5891 1092 24877 2233.5 6098 17477.5 3408 10070.5 511 98]


In [69]:
# Step 3
# determining the a,b,c,d values from xdata and ydata # needs to be automated

#copy the above x,y values into the following website and calculate X50 and get a,b,c,d values from formula
#https://www.aatbio.com/tools/four-parameter-logistic-4pl-curve-regression-online-calculator

a = 127.1738
b = 1.6959
c = 0.2326
d = 32667.2563

In [70]:
# Step 4
# function for calculating x based on y, a,b,c, and d

#function for calculating concentrations from y values
#https://github.com/Tom-Alexander/regression-js/issues/96

def func_find_x (y, a, b, c, d):
  x = c*((((a-d)/(y-d)) -1)**(1/b))
  return x

In [71]:
# Step 5 to calulate the x_test

# test data calculations

x_test = func_find_x(ytest,a,b,c,d)
print('x_test: ', x_test)

x_test:  [0.036222988165252296 (-2.489526331789298+8.603557888330245j)
 (-2.4259449969135156+8.383827055104659j) 1.3479808500945185
 (-2.4157696773654056+8.34866215259025j)
 (-2.4678354360959367+8.528596288460992j)
 (-2.4259449969135156+8.383827055104659j)
 (-2.4057089107051466+8.313893133577581j)
 (-2.4466479072868523+8.455374274983186j)
 (-2.4259449969135156+8.383827055104659j) 2.9538124353916912
 (-11.823031376347641+40.859232362022226j) 0.12520537022094255
 1.2546614718677265 0.459452841959041 0.589031740405578 17.55741284303603
 1.5179275089471393 0.42978135154254554 6.072048357224703
 0.1589784148289986 0.04771442832489203 0.09403407004848402
 0.029739865558942494 0.4598643711051261 0.04816232008128662
 0.09645134676717923 0.25157440712462836 0.06401327297728737
 0.1433469128325522 0.01708525036177118
 (-0.0010313862724883706+0.0035643694092623354j)]


In [72]:
# Step 6
# adding the x_test values to test dataframe

test_df['IgG Concentrations_interpolated'] = x_test
test_df

print(test_df.columns)
# write the op into output dir
test_df.to_csv(analysis_dir_name_path + '/' + opdir_name + '/' + sample_result_file_name.split('.', 1)[0] + '_conc_interpolated.csv')

############################################################################################################################################################

Index(['SampleID', 'CollectDate', 'Well', 'Type', 'DF', 'SCV2SP', 'SCV2NP',
       'OC43', 'HKU1', '229E', 'NL63', 'IgG Concentrations',
       'IgG Concentrations_interpolated'],
      dtype='object')


<ipython-input-72-18c4a3d5d323>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['IgG Concentrations_interpolated'] = x_test


In [73]:
# Step 7

test_df['final'] = test_df['DF']*test_df['IgG Concentrations_interpolated']


print(test_df.columns)


Index(['SampleID', 'CollectDate', 'Well', 'Type', 'DF', 'SCV2SP', 'SCV2NP',
       'OC43', 'HKU1', '229E', 'NL63', 'IgG Concentrations',
       'IgG Concentrations_interpolated', 'final'],
      dtype='object')


<ipython-input-73-92dae2aff4f9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['final'] = test_df['DF']*test_df['IgG Concentrations_interpolated']


In [74]:
# step 8 write the final output file
# write the op into output dir
# Remove 'SampleID','Well', 'Type','IgG Concentrations'
test_df = test_df.drop(['SampleID','Well', 'Type','IgG Concentrations'], axis=1)
test_df.to_csv(analysis_dir_name_path + '/' + opdir_name + '/' + sample_result_file_name.split('.', 1)[0] + '_final_output.csv')

In [75]:
from scipy.optimize import curve_fit

p0 = [max(ydata), np.median(xdata),1,min(ydata)] # this is an mandatory initial guess
print(p0)
popt, pcov = curve_fit(func_find_x, xdata, ydata,p0, method='lm') ## Updated method from 'dogbox' to 'lm' 9.30.2021
print(popt)

[31760, 0.1003125, 1, 717]
[3.176000e+04 1.003125e-01 1.000000e+00 7.170000e+02]


<ipython-input-70-3e4fdf3e02c4>:8: RuntimeWarning: invalid value encountered in power
  x = c*((((a-d)/(y-d)) -1)**(1/b))


In [76]:
# Initial approximations of parameters
A = np.amin(ydata)#np.amax([np.amin(y_meas)]) # Min asymptote
D = np.amax(ydata) # Max asymptote
B = (D - A) / np.amax(xdata) # Steepness
C = np.amax(xdata) / 2
print(A,B,C,D)

717 29012.14953271028 0.535 31760


In [77]:
# testing
# finding a,b,c,d

def func_find_x (y, a, b, c, d):
  x = c*((((a-d)/(y-d)) -1)**(1/b))
  return x

def residuals(p, y, x):
    """Deviations of data from fitted 4PL curve"""
    A,B,C,D = p
    err = x-func_find_x(y, A, B, C, D)
    return err

In [80]:
from scipy.optimize import leastsq

a,b,c,d = 253,1.6,0.2,30401.5
# Initial guess for parameters
p0 = np.array([0,1,1,1])
#p0 = [0,1,1,1]
'''a =  253
b =  0.9916780422018651
c =  15074.25
d =  30401.5'''

# Fit equation using least squares optimization
plsq = leastsq(residuals, p0, args=(ydata, xdata))
print('plsq' , plsq)

TypeError: ignored

error: ignored

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
import numpy as np
import numpy.random as npr

def logistic4(x, A, B, C, D):
    """4PL lgoistic equation."""
    return ((A-D)/(1.0+((x/C)**B))) + D


x = np.linspace(0,20,20)
print(x)
A,B,C,D = 0.5,2.5,8,7.3
y_true = logistic4(x, A, B, C, D)
print('y_true: ', y_true)
y_meas = y_true + 0.2*npr.randn(len(x))
print('y_meas: ', y_meas)

print(len(x))
print(len(y_meas))

print(x)
print(y_meas)

# Initial approximations of parameters
A = np.amin(y_meas)#np.amax([np.amin(y_meas)]) # Min asymptote
D = np.amax(y_meas) # Max asymptote
B = (D - A) / np.amax(x) # Steepness
C = np.amax(x) / 2 # Inflection Point, conc at which y = (D - A) / 2
print(A, B, C, D)

In [ ]:
test_df.head()

In [ ]:
x_min, x_max = np.amin(ydata), np.amax(ydata)

A = x_min
D = x_max
B = (D-A)/np.amax(ydata)


#C = np.amax(x) / 2
C = (D-A)/2

print('a = ', A)
print('b = ', B)
print('c = ', C)
print('d = ', D)

'''
a = 127.1738
b = 1.6959
c = 0.2326
d = 32667.2563
'''

In [ ]:
'''for i in ytest:
  y = func_find_x(i,a,b,c,d)
  print(i,'\t', y)'''

In [ ]:
import matplotlib.pyplot as plt

plt.plot(xdata, ydata,'--',color='green')#, 'o', xnew, f(xnew), '-', xnew, f2(xnew), '--')
plt.xlabel('xdata')
plt.ylabel('ydata')
plt.title('Sigmoidal 4PL curve')
plt.scatter(xdata, ydata, color='red', marker='*',s=100, linewidth=1)
plt.show()

In [ ]:
def func_find_x (y, a, b, c, d):
  x = c*((((a-d)/(y-d)) -1)**(1/b))
  return x


a = 589.0309
b = 2.0813
c = 0.1824
d = 32081.14
y = 17560

x = func_find_x(y, a, b, c, d)
print(x)

 # r code for inverse function https://weightinginbayesianmodels.github.io/poctcalibration/calib_tut4_curve_background.html

In [ ]:
def logistic4(x, A, B, C, D):
    """4PL lgoistic equation."""
    return ((A-D)/(1.0+((x/C)**B))) + D

a = 589.0309
b = 2.0813
c = 0.1824
d = 32081.14
#y = 17560
X = 0.19658728749938234

Y = logistic4(X, a, b, c, d)
print(Y)

In [ ]:
a1 = 421.3913
b1 = 1.7457
c1 = 0.22421
d1 = 33455.1869
y1 = 1



In [ ]:
p0 = [max(ydata), np.median(xdata),1,min(ydata)]
p0

In [ ]:
x = [10, 5, 2.5, 1.25, 0.625, 0.312, 0.156,0];
y = [41564.9, 21531.9, 15086.9, 9249, 3175.9, 1781.9, 1320.9, 182.9];
y2 = [1308.9, 2514.9, 4797.9];

x2 = interp1(y, x, y2, 'linear');

figure(1)
plot(x, y, '-g')
hold on
plot(x2, y2, 'bp')
hold off
grid
legend('Data', 'Interpolated Points', 'Location', 'NW')

In [ ]:
p0 = [max(ydata), np.median(xdata),1,min(ydata)] # this is a mandatory initial guess
p0

In [ ]:
'''https://weightinginbayesianmodels.github.io/poctcalibration/calib_tut4_curve_ocon.html
example for inverse function
x=c(a−dy−d−1)1/b'''

In [ ]:
from scipy.optimize import curve_fit
def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0))) + b
    return (y)

p0 = [max(ydata), np.median(xdata),1,min(ydata)] # this is a mandatory initial guess

popt, pcov = curve_fit(sigmoid, xdata, ydata,p0, method='dogbox')
popt
pcov

In [ ]:
def func(x, a, b, c, d):
    return a / (1. + np.exp(-c * (x - d))) + b

popt, pcov = curve_fit(func, xdata, ydata)
print(popt)
print(pcov)

In [ ]:
from scipy.optimize import curve_fit

def sigmoid(x, k, x0):

    return 1.0 / (1 + np.exp(-k * (x - x0)))

# Parameters of the true function
n_samples = 100
true_x0 = 15
true_k = 1.5
sigma = 0.2

# Build the true function and add some noise
x = np.linspace(0, 30, num=n_samples)
y = sigmoid(x, k=true_k, x0=true_x0)
y_with_noise = y + sigma * np.random.randn(n_samples)

# Sample the data from the real function (this will be your data)
some_points = np.random.choice(100, size=30)  # take 30 data points
xdata = x[some_points]
ydata = y_with_noise[some_points]

# Fit the curve
popt, pcov = curve_fit(sigmoid, xdata, ydata)
estimated_k, estimated_x0 = popt

# Plot the fitted curve
y_fitted = sigmoid(x, k=estimated_k, x0=estimated_x0)

# Plot everything for illustration
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x, y_fitted, '--', label='fitted')
ax.plot(x, y, '-', label='true')
ax.plot(xdata, ydata, 'o', label='samples')

ax.legend()

In [ ]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
from scipy.optimize import leastsq

def logistic4(x, A, B, C, D):
    """4PL lgoistic equation."""
    return ((A-D)/(1.0+((x/C)**B))) + D

def residuals(p, y, x):
    """Deviations of data from fitted 4PL curve"""
    A,B,C,D = p
    err = y-logistic4(x, A, B, C, D)
    return err

def peval(x, p):
    """Evaluated value at x with current parameters."""
    A,B,C,D = p
    return logistic4(x, A, B, C, D)

# Make up some data for fitting and add noise
# In practice, y_meas would be read in from a file
x = np.linspace(0,20,20)
print(x)
A,B,C,D = 0.5,2.5,8,7.3
y_true = logistic4(x, A, B, C, D)
print('y_true: ', y_true)
y_meas = y_true + 0.2*npr.randn(len(x))
print('y_meas: ', y_meas)

# Initial guess for parameters
p0 = [0, 1, 1, 1]

# Fit equation using least squares optimization
plsq = leastsq(residuals, p0, args=(y_meas, x))
print('plsq' , plsq)

# Plot results
plt.plot(x,peval(x,plsq[0]),x,y_meas,'o',x,y_true)
plt.title('Least-squares 4PL fit to noisy data')
plt.legend(['Fit', 'Noisy', 'True'], loc='upper left')
for i, (param, actual, est) in enumerate(zip('ABCD', [A,B,C,D], plsq[0])):
    plt.text(10, 3-i*0.5, '%s = %.2f, est(%s) = %.2f' % (param, actual, param, est))
plt.savefig('logistic.png')

In [ ]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
from scipy.optimize import leastsq

def logistic4(x, A, B, C, D):
    """4PL lgoistic equation."""
    return ((A-D)/(1.0+((x/C)**B))) + D

def residuals(p, y, x):
    """Deviations of data from fitted 4PL curve"""
    A,B,C,D = p
    err = y-logistic4(x, A, B, C, D)
    return err

def peval(x, p):
    """Evaluated value at x with current parameters."""
    A,B,C,D = p
    return logistic4(x, A, B, C, D)

# Make up some data for fitting and add noise
# In practice, y_meas would be read in from a file
x = np.linspace(0,20,20)
print(x)
A,B,C,D = 0.5,2.5,8,6.6
y_true = logistic4(x, A, B, C, D)
print('y_true: ', y_true)
y_meas = y_true + 0.2*npr.randn(len(x))
print('y_meas: ', y_meas)

# Initial guess for parameters
p0 = [0, 1, 1, 1]

# Fit equation using least squares optimization
plsq = leastsq(residuals, p0, args=(y_meas, x))
print('plsq' , plsq)

# Plot results
plt.plot(x,peval(x,plsq[0]),x,y_meas,'o',x,y_true)
plt.title('Least-squares 4PL fit to noisy data')
plt.legend(['Fit', 'Noisy', 'True'], loc='upper left')
for i, (param, actual, est) in enumerate(zip('ABCD', [A,B,C,D], plsq[0])):
    plt.text(10, 3-i*0.5, '%s = %.2f, est(%s) = %.2f' % (param, actual, param, est))
plt.savefig('logistic.png')

In [ ]:
# good discussion here:  http://stackoverflow.com/questions/4308168/sigmoidal-regression-with-scipy-numpy-python-etc
# curve_fit() example from here: http://permalink.gmane.org/gmane.comp.python.scientific.user/26238
# other sigmoid functions here: http://en.wikipedia.org/wiki/Sigmoid_function

import numpy as np
import pylab
from scipy.optimize import curve_fit

def sigmoid(x, x0, k):
     y = 1 / (1 + np.exp(-k*(x-x0)))
     return y

xdata = np.array([0.0,   1.0,  3.0, 4.3, 7.0,   8.0,   8.5, 10.0, 12.0])
ydata = np.array([0.01, 0.02, 0.04, 0.11, 0.43,  0.7, 0.89, 0.95, 0.99])

popt, pcov = curve_fit(sigmoid, xdata, ydata)
print( popt)
print(pcov)

x = np.linspace(-1, 15, 50)
y = sigmoid(x, *popt)

pylab.plot(xdata, ydata, 'o', label='data')
pylab.plot(x,y, label='fit')
pylab.ylim(0, 1.05)
pylab.legend(loc='best')
pylab.show()

In [ ]:
def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0))) + b
    return (y)

xdata = x
ydata = y_meas

print(max(ydata))
print(np.median(xdata))
print(min(ydata))
'''p0 = [max(ydata), np.median(xdata),1,min(ydata)] # this is an mandatory initial guess
popt, pcov = curve_fit(sigmoid, xdata, ydata,p0, method='lm') ## Updated method from 'dogbox' to 'lm' 9.30.2021
print( popt)
print(pcov)'''

In [ ]:
import numpy as np
import pylab
from scipy.optimize import curve_fit

def sigmoid(x, x0, k):
     y = 1 / (1 + np.exp(-k*(x-x0)))
     return y

print(x)
print(y_meas)

popt, pcov = curve_fit(sigmoid, x, y_meas)
print( popt)
print(pcov)

In [ ]:
y_true = logistic4(0, 0.5,2.5,8,7.3)
y_true

In [ ]:
'''print(x)
print(len(x))
y_meas = 0.5 + 0.2*npr.randn(20)
print(y_meas)'''
print(0.2*npr.randn(20))

'''y_meas = 0.5 + 0.2*npr.randn(len(x))
y_meas'''

In [ ]:
#https://www.medcalc.org/manual/nonlinearregression-example.php

In [ ]:
# example https://ipython-books.github.io/93-fitting-a-function-to-data-with-nonlinear-least-squares/
#1.  Let's import the usual libraries:

import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
%matplotlib inline


We define a logistic function with four parameters:

fa,b,c,d(x)=a1+exp(−c(x−d))+b


In [ ]:
def f(x, a, b, c, d):
    return a / (1. + np.exp(-c * (x - d))) + b

In [ ]:
#3.  Let's define four random parameters:

a, c = np.random.exponential(size=2)
b, d = np.random.randn(2)
print(a)
print(b)
print(c)
print(d)

In [ ]:
#4.  Now, we generate random data points by using the sigmoid function and adding a bit of noise:

n = 100
x = np.linspace(-10., 10., n)
y_model = f(x, a, b, c, d)
print(y_model)
y = y_model + a * .2 * np.random.randn(n)
print(y)

In [ ]:
#5.  Here is a plot of the data points, with the particular sigmoid used for their generation (in dashed black):

fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(x, y_model, '--k')
ax.plot(x, y, 'o')

In [ ]:
'''6.  We now assume that we only have access to the data points and not the underlying generative function. These points could have been obtained during an experiment. By looking at the data, the points appear to approximately follow a sigmoid, so we may want to try to fit such a curve to the points. That's what curve fitting is about. SciPy's curve_fit() function allows us to fit a curve defined by an arbitrary Python function to the data:
'''
(a_, b_, c_, d_), _ = opt.curve_fit(f, x, y)
print(a_)
print(b_)
print(c_)
print(d_)

In [ ]:
#7.  Now, let's take a look at the fitted sigmoid curve:

y_fit = f(x, a_, b_, c_, d_)
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(x, y_model, '--k')
ax.plot(x, y, 'o')
ax.plot(x, y_fit, '-')

In [ ]:
import urllib.request
page = urllib.request.urlopen("https://www.aatbio.com/tools/four-parameter-logistic-4pl-curve-regression-online-calculator")
print (page.read())

In [ ]:
# activate R magic
%load_ext rpy2.ipython

In [ ]:

library(drc)
library(ggplot2)
library(magrittr)
library(Cairo)

data_synthetic = data.frame(c(0.1,1,2,3,10), c(0,20,40,60, 80))
names(data_synthetic) <- c('concentration', 'inhibition')

In [ ]:
%load_ext rpy2.ipython


In [ ]:
!pip install rpy2==3.0.0
#Conversion 'py2rpy' not defined for objects of type '<class 'str'>'
#This is a problem with newer versions. You must downgrade rpy: run in a cell:
#

In [ ]:
%%R
x<-1:5
x

In [ ]:
%%R
#https://towardsdatascience.com/drug-dose-response-data-analysis-5d7d336ad8e9
library(drc)
library(ggplot2)
library(magrittr)
#library(Cairo)

data_synthetic = data.frame(c(0.1,1,2,3,10), c(0,20,40,60, 80))
names(data_synthetic) <- c('concentration', 'inhibition')

In [ ]:
%%R
plot(data_synthetic,
     log='x',
     xlab = 'Drug concentration (microM)',
     ylab= 'Inhibition')

In [ ]:
%%R
fitted_curve <- drm(formula = inhibition ~ concentration,
                    data = data_synthetic,
                    fct = LL.4())
plot(fitted_curve,
     log='x',
     xlab = 'Drug concentration (microM)',
     ylab= 'Inhibition')

In [ ]:
%%R
E0 <- fitted_curve$coefficients[2]
Einf <- fitted_curve$coefficients[3]
EC50 <- fitted_curve$coefficients[4]
H <-  -fitted_curve$coefficients[1]
IC50 <- exp(log((((Einf - E0) / (50 - E0) ) - 1) * EC50**(-H))/ (-H))

efficacy_metrics <- data.frame(c(E0, Einf, EC50, H, IC50))
rownames(efficacy_metrics) <- c('E0', 'Einf', 'EC50', 'H', 'IC50')
colnames(efficacy_metrics) <- 'Value'
efficacy_metrics

In [ ]:
import pandas as pd
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#https://gist.github.com/yannabraham/5f210fed773785d8b638

In [ ]:
def ll4(x,b,c,d,e):
    '''This function is basically a copy of the LL.4 function from the R drc package with
     - b: hill slope
     - c: min response
     - d: max response
     - e: EC50'''
    return(c+(d-c)/(1+np.exp(b*(np.log(x)-np.log(e)))))

def pDose(x):
    '''This is just a helper function, to compute easily log transformed concentrations used in drug discovery'''
    return(-np.log10(1e-6*x))

In [ ]:
params = [{'compound':'A', 'b':1, 'c':0, 'd':100, 'e':0.4,'startDose':10, 'nDose':8, 'dilution':3},
          {'compound':'B', 'b':0.7, 'c':0, 'd':86, 'e':1.3,'startDose':30, 'nDose':8, 'dilution':3},
          {'compound':'C', 'b':2, 'c':24, 'd':152, 'e':0.02,'startDose':3, 'nDose':8, 'dilution':3}]
paramsCompound = [item['compound'] for item in params]

In [ ]:
drData=[]
for curve in params:
    # generate base curve
    curData = pd.DataFrame(data={'compound':curve['compound'],
                                 'dose':curve['startDose']/np.power(curve['dilution'],range(curve['nDose']))})
    curData['logDose'] = pDose(curData.dose)
    curData['response'] = curData.dose.apply(lambda x: ll4(x,*[curve[i] for i in ['b','c','d','e']]))
    # generate replicates
    repData = []
    for i in range(5):
        rep = curData
        rep.response += 0.25*np.random.normal(len(rep.response))
        repData.append(rep.copy())
    repData = pd.concat(repData)
    drData.append(repData)
# assemble data
drData = pd.concat(drData)
drData.head()

In [ ]:
fitTable = pd.DataFrame(fitData).set_index('compound')
fitTable

In [ ]:
fitTable = pd.DataFrame(fitData).set_index('compound')
paramTable = pd.DataFrame(params).set_index('compound')[['b','c','d','e']]
paramTable.columns = ['ref_'+i for i in paramTable.columns]
fitTable.join(paramTable)